# Previsão da probabilidade de inadimplência

Este notebook tem como objetivo aplicar o modelo final treinado para estimar a probabilidade de inadimplência na base de dados que não possui a variável alvo. Nele, o modelo salvo é carregado, os dados inéditos passam pelos mesmos passos de pré-processamento utilizados no treinamento, e em seguida são geradas as previsões de probabilidade e classe. Por fim, os resultados são exportados para uso posterior. Vale lembrar que este notebook é voltado apenas para a etapa de inferência, ou seja, não realiza nenhum tipo de re-treinamento do modelo. **É necessário executar os notebooks anteriores para chegar até aqui**.


In [1]:
import pickle
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

import warnings

# remoção segura para deixar o log mais limpo
warnings.filterwarnings("ignore", message="X does not have valid feature names") 

Pré-processamento

In [2]:
# Leitura do dataset
dataset_features_v1 = pd.read_csv('../data/processed/dataset_features_v1.csv')

# Conversão dos tipos numéricos e categóricos
dataset_features_v1 = dataset_features_v1.astype({
    'ID_CLIENTE': 'int64',
    'VALOR_A_PAGAR': 'float64',
    'TAXA': 'float64',
    'RENDA_MES_ANTERIOR': 'float64',
    'NO_FUNCIONARIOS': 'int64',
    'FLAG_PF': 'int64',
    'DIAS_ATRASO': 'int64',
    'TARGET_INADIMPLENCIA': 'int64',
    'DIAS_ADIANTAMENTO': 'int64',
    'TEMPO_DE_CASA_MESES': 'int64',
    'PRAZO_PAGAMENTO_DIAS': 'int64',
    'MES_SAFRA': 'int32',
    'INADIMPLENCIAS_ANTERIORES': 'int64',
    'SEGMENTO_INDUSTRIAL': 'category',
    'DOMINIO_EMAIL': 'category',
    'PORTE': 'category',
    'CEP_2_DIG': 'category',
})

# Conversão de datas
dataset_features_v1['SAFRA_REF'] = pd.to_datetime(
    dataset_features_v1['SAFRA_REF'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_EMISSAO_DOCUMENTO'] = pd.to_datetime(
    dataset_features_v1['DATA_EMISSAO_DOCUMENTO'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_PAGAMENTO'] = pd.to_datetime(
    dataset_features_v1['DATA_PAGAMENTO'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_VENCIMENTO'] = pd.to_datetime(
    dataset_features_v1['DATA_VENCIMENTO'], format='%Y-%m-%d', errors='coerce'
)

dataset_features_v1['DATA_CADASTRO'] = pd.to_datetime(
    dataset_features_v1['DATA_CADASTRO'], format='%Y-%m-%d', errors='coerce'
)

# Eliminar colunas irrelevantes
cols_drop = [
    'ID_CLIENTE', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO',
    'DATA_VENCIMENTO', 'DATA_CADASTRO', 'SAFRA_REF',
    'DIAS_ATRASO', 'DIAS_ADIANTAMENTO', # Colunas que só são conhecidas depois do treinamento. São removidas para evitar data leakage
]

df = dataset_features_v1.drop(columns=cols_drop)


# Aplicando One-Hot Encoding em variáveis categóricas de baixa cardinalidade
df = pd.get_dummies(df, columns=['SEGMENTO_INDUSTRIAL', 'PORTE', 'DOMINIO_EMAIL'], drop_first=True)

# Aplicando Frequency Encoding para variável com alta cardinalidade 
freq_map = df['CEP_2_DIG'].value_counts(normalize=True)
df['CEP_2_DIG'] = df['CEP_2_DIG'].map(freq_map)

# Importante não adionar a variável target, resultaria em data leackage e consequentemente overfitting
X = df.drop(columns='TARGET_INADIMPLENCIA')
y = df['TARGET_INADIMPLENCIA']

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [3]:
# === Hiperparâmetros ótimos para Random Forest ===
melhores_params = {
    'bootstrap': False,
    'max_depth': None,
    'max_features': 'log2',
    'min_samples_leaf': 2,
    'min_samples_split': 5,
    'n_estimators': 300,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': 1  # ativa logs do Random Forest durante o fit
}

# === Pipeline com SMOTE + Escalonamento + Random Forest ===
pipeline = ImbPipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(**melhores_params))
])

# === Treinamento final no conjunto completo ===
print("\n=== Treinando o modelo final com todos os dados ===")
pipeline.fit(X, y)

# === Salvar modelo treinado ===
with open('../data/processed/final_random_forest_structure.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

print("Modelo salvo em '../data/processed/final_random_forest_structure.pkl'")



=== Treinando o modelo final com todos os dados ===


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    9.7s finished


Modelo salvo em '../data/processed/final_random_forest_structure.pkl'


Persistindo dados de probabilidade de inadimplência

In [1]:
import pickle

# Abrindo e carregando o conteúdo
with open('../data/processed/resultados_grid_search_cv.pkl', 'rb') as arquivo:
    objeto = pickle.load(arquivo)

# Exibindo (opcional)
print(objeto)


{'XGBoost': {'Melhores parâmetros': {'clf__colsample_bytree': 0.6, 'clf__gamma': 0, 'clf__learning_rate': 0.1, 'clf__max_depth': 10, 'clf__min_child_weight': 1, 'clf__n_estimators': 300, 'clf__subsample': 0.8}, 'ROC AUC (validação)': 0.9600524212116186, 'ROC AUC (teste)': 0.9685975419379949, 'Classification Report': {'0': {'precision': 0.977415124581087, 'recall': 0.981275599765945, 'f1-score': 0.97934155777794, 'support': 13672.0}, '1': {'precision': 0.7521781219748306, 'recall': 0.7148114075436982, 'f1-score': 0.7330188679245283, 'support': 1087.0}, 'accuracy': 0.9616505183278, 'macro avg': {'precision': 0.8647966232779588, 'recall': 0.8480435036548216, 'f1-score': 0.8561802128512341, 'support': 14759.0}, 'weighted avg': {'precision': 0.9608264246804838, 'recall': 0.9616505183278, 'f1-score': 0.9611998975116172, 'support': 14759.0}}, 'Confusion Matrix': [[13416, 256], [310, 777]]}, 'RandomForest': {'Melhores parâmetros': {'clf__bootstrap': False, 'clf__max_depth': None, 'clf__max_fea